In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!git clone https://github.com/NAIST-SE/DevGPT

fatal: destination path 'DevGPT' already exists and is not an empty directory.


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import json
import os
import openai

In [ ]:
import os
import json

def remove_duplicates_and_extract(json_directory, output_file_path):
    unique_conversations = []

    for filename in os.listdir(json_directory):
        if filename.endswith('.json'):
            file_path = os.path.join(json_directory, filename)

            with open(file_path, 'r') as json_file:
                data = json.load(json_file)

            for source in data.get('Sources', []):
                for chat in source.get('ChatgptSharing', []):
                    if "NumberOfPrompts" in chat and chat["NumberOfPrompts"] == 1:
                        prompt = chat["Conversations"][0].get("Prompt", "")
                        answer = chat["Conversations"][0].get("Answer", "")
                        if "[CODE_BLOCK_0]" not in answer:
                            conversation = {"Prompt": prompt, "Answer": answer}
                            if conversation not in unique_conversations:
                                unique_conversations.append(conversation)

    with open(output_file_path, 'w') as json_file:
        json.dump(unique_conversations, json_file, indent=4)

    print(f"All unique prompts and answers saved to {output_file_path}")

# Usage
json_directory = '/content/drive/MyDrive/DevGPT/snapshot_20230831'
output_file_path = "/content/drive/MyDrive/SE_Project/samp6_unique.json"
remove_duplicates_and_extract(json_directory, output_file_path)


All unique prompts and answers saved to /content/drive/MyDrive/SE_Project/samp6_unique.json


In [ ]:
import json
import re

def contains_chinese_or_japanese(text):
    # Use regular expression to check if the text contains Chinese or Japanese characters
    chinese_japanese_pattern = re.compile('[\u4e00-\u9fa5ぁ-んァ-ヶ]')
    return bool(chinese_japanese_pattern.search(text))

def preprocess_json(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        data = json.load(infile)

        # Filter out entries where either 'Prompt' or 'Answer' contains Chinese or Japanese characters
        filtered_data = [item for item in data if not (contains_chinese_or_japanese(item['Prompt']) or contains_chinese_or_japanese(item['Answer']))]

    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(filtered_data, outfile, ensure_ascii=False, indent=4)


if __name__ == "__main__":
    input_file_path = '/content/drive/MyDrive/SE_Project/samp6_unique.json'  # Path to the uploaded JSON file
    output_file_path = '/content/drive/MyDrive/SE_Project/samp6_final.json'

    preprocess_json(input_file_path, output_file_path)


In [ ]:
import openai
import json

# Set your OpenAI API key
openai.api_key = 'OPENAI_KEY'

# Read prompts from JSON file
with open('/content/drive/MyDrive/SE_Project/samp6_unique.json', 'r') as file:
    data = json.load(file)

# List to store new responses
new_responses = []

# Loop through each prompt and make API calls
for item in data:
    prompt = item["Prompt"]

    try:
        response = openai.Completion.create(
            model="gpt-3.5-turbo-1106",  # Updated model
            prompt=prompt,
            max_tokens=1500  # You can adjust this parameter based on the desired response length
        )
    except Exception as e:
        print(f"Error generating response for prompt: {prompt}")
        print(e)
        continue

    # Extract the generated response
    generated_response = response['choices'][0]['text'].strip()

    # Store the new response in the same format
    new_responses.append({
        "Prompt": prompt,
        "Answer": generated_response
    })

# Save the new_responses to a JSON file
output_file_path = "/content/drive/MyDrive/SE_Project/samp6_unique_openai.json"
with open(output_file_path, 'w') as file:
    json.dump(new_responses, file, indent=4)

print(f"New responses saved to {output_file_path}")
